# Vaccination data pre-processing

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import nafot
from tqdm import tqdm, tqdm_notebook
import networkx as nx
import matplotlib.dates
import pickle
%matplotlib inline

### vaccination data

In [76]:
# vaccination_data = pd.read_csv('../../Data/vaccination_data/vaccinated_patients_raw.csv')
# vaccination_data.drop(['Unnamed: 0'], axis=1, inplace=True)
# vaccination_data.columns = ['random_ID', 'vac_date', 'vac_season', 'birth_year', 'clinic_code', 'stat_area_id']
# vaccination_data['vac_date'] = pd.to_datetime(vaccination_data['vac_date'])
# vaccination_data['vac_day_of_season'] = vaccination_data.vac_date.apply(lambda date: (date - pd.datetime(date.year if date.month>5 else date.year-1, 6, 1)).days)
# vaccination_data = vaccination_data[['random_ID', 'birth_year', 'vac_season', 'vac_date', 'vac_day_of_season', 'clinic_code', 'stat_area_id']]
# vaccination_data.to_csv('../../Data/vaccination_data/vaccinated_patients.csv', index=False)

### Vaccination coverage data

In [90]:
vacc_prop = pd.read_csv('../../Data/vaccination_data/clinics_vaccinated_proportion_each_season.csv')
vacc_prop.drop(['Unnamed: 0'], axis=1, inplace=True)
# Remove incoplete seasons
vacc_prop = vacc_prop[~vacc_prop.VaccinationSeason.isin([2007,2018])].copy()
vacc_prop.columns = ['clinic_code', 'vaccination_season', 'vaccination_coverage']

In [91]:
# load stat_area-clinic data
clinics_stat_areas = pd.read_csv('../../Data/vaccination_data/clinics_with_stat_area.csv')

# Add stat_area to vacc_prop
vacc_prop = vacc_prop.merge(clinics_stat_areas[['clinic_code', 'stat_area_id']], on='clinic_code')

# Add subdist column (code and name)
vacc_prop = vacc_prop.merge(nafot.gdf[['SubDistrictCode']], left_on='stat_area_id', right_index=True)
vacc_prop['subdist'] = vacc_prop.SubDistrictCode.map(lambda code: nafot.subdistricts_names[code])

# Save
vacc_prop.to_csv('../../Data/vaccination_data/vacc_coverage.csv', index=False)